## Dependencies

In [1]:
# Confirmed needed dependencies
import zillow
import random
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim


# Research how to make config.py truly invisible on GitHub Repo
# from config import zwsid
# Paste in your zwsid in the meantime using the following line
# zwsid = ""



# In README give instructions on setting config file and needed variables for project

## File inputs/outputs

In [2]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

In [3]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [4]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [5]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 1000        #### Set number of lat,lon points to generate

In [6]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points = points+1
#####this can be written more succinctly as points += 1

In [7]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

1000

In [8]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map

Troy is working on this as a way to verify good random sampling of coordinate points and area selection, and as a way to get familar with plotting on maps


In [20]:
gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

#gmap.plot(latitudes, longitudes, 'cornflowerblue', edge_width=10)
gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"], '#3B0B39', size=10, marker=False)
#gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"] , 'k', marker=True)
#gmap.heatmap(, heat_lngs)

gmap.draw("./Visuals/mymap.html")

## Convert Coordinates to Residential Addresses

In [10]:
#########################
##### Yuta's Blocks #####
#########################


##### General Analysis on the Geopy Nominatim Method Below #####
# - You can filter the address by type='house', but some houses are labeled with type='yes'
# - 'Yes' type means means it's an unlabeled building.
# - Quite a bit of 'yes' type buildings actually hit a residential building, so we may get a high percentage of
#   residential buildings if we filter by 'house' or 'yes'.
# - Out of roughly 100 requests, around 70% will result in either 'house' or 'yes' type.
# - I can't seem to find the limit of API request, but we are throttled to 1 request per second.

# Testing Geopy Nominatim API Response


geopy.geocoders.options.default_user_agent = "my-application"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

# Params 1 - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== #01 - Home Response:")
response_1 = requests.get(url, params=params_1).json()
pp.pprint(response_1)
print("\n" + "="*60 + "\n")

===== #01 - Home Response:
{   'address': {   'country': 'USA',
                   'country_code': 'us',
                   'county': 'Williamson County',
                   'hamlet': 'Pond Springs',
                   'house_number': '12126',
                   'neighbourhood': "Hunter's Chase",
                   'postcode': '78729',
                   'road': 'Hunters Chase Drive',
                   'state': 'Texas',
                   'suburb': 'Jollyville'},
    'addresstype': 'place',
    'boundingbox': ['30.440962', '30.441162', '-97.777478', '-97.777278'],
    'category': 'place',
    'display_name': "12126, Hunters Chase Drive, Hunter's Chase, Jollyville, "
                    'Pond Springs, Williamson County, Texas, 78729, USA',
    'importance': '0',
    'lat': '30.441062',
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
               'https://osm.org/copyright',
    'lon': '-97.777378',
    'name': None,
    'osm_id': '3825454876',
    'osm_type': 'node',
 

In [11]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

,lat,lon
0,30.167536,-97.774422
1,30.207079,-97.735414
2,30.296498,-97.883496
3,30.236841,-97.837287
4,30.308969,-97.625322


In [12]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

# Might not need to do this step, see below

In [13]:
# Loop Request API and append to create new dataframe
query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
# Verify this field is unneeded, then delete if not
# aType = []
lat = []
lon = []
neighborhood = []
# Verify this field is unneeded, then delete if not
# addType = []

counter = 1
numRequests = len(lats)

print(f"Processing {numRequests} Requests...")


# Don't actually need to zip lat and lons for this
# can use "for lat_lon in latlon_df.itertuples():"
# Check into this and see if you can cut out the zip part
for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }
# Check to see if you can do multiple queries at the same time to make this faster
# i.e. send batch requests/list of queries rather than loop of single queries
    response = requests.get(query_url, params=params).json()
    
    if response['type'] == 'house' or response['type'] == 'yes':
        postcode.append(response['address']['postcode'])
#         Do we need to store aType?
#         aType.append(response['type'])
        lat.append(response['lat'])
        lon.append(response['lon'])
    
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
            
# Verify this field is unneeded, then delete if not
#         try:
#             addType.append(response['addresstype'])
#         except (KeyError, IndexError):
#             addType.append("NA")


# Printing to console is good for development to see what is happening, but takes a long time.  
# Instead of printing each line to the console, can you store these in two lists (success and failure) 
# that we could run len() on to get counts for stats later?
        print(f"Processed Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1

Processing 1000 Requests...
Wrong Type - Skipped Record 1 of 1000.
Processed Record 2 of 1000.
Processed Record 3 of 1000.
Processed Record 4 of 1000.
Wrong Type - Skipped Record 5 of 1000.
Processed Record 6 of 1000.
Wrong Type - Skipped Record 7 of 1000.
Processed Record 8 of 1000.
Processed Record 9 of 1000.
Processed Record 10 of 1000.
Wrong Type - Skipped Record 11 of 1000.
Wrong Type - Skipped Record 12 of 1000.
Processed Record 13 of 1000.
Processed Record 14 of 1000.
Processed Record 15 of 1000.
Processed Record 16 of 1000.
Wrong Type - Skipped Record 17 of 1000.
Processed Record 18 of 1000.
Processed Record 19 of 1000.
Wrong Type - Skipped Record 20 of 1000.
Processed Record 21 of 1000.
Processed Record 22 of 1000.
Processed Record 23 of 1000.
Processed Record 24 of 1000.
Wrong Type - Skipped Record 25 of 1000.
Wrong Type - Skipped Record 26 of 1000.
Processed Record 27 of 1000.
Wrong Type - Skipped Record 28 of 1000.
Processed Record 29 of 1000.
Processed Record 30 of 1000.
P

Processed Record 248 of 1000.
Processed Record 249 of 1000.
Processed Record 250 of 1000.
Processed Record 251 of 1000.
Processed Record 252 of 1000.
Processed Record 253 of 1000.
Processed Record 254 of 1000.
Wrong Type - Skipped Record 255 of 1000.
Processed Record 256 of 1000.
Wrong Type - Skipped Record 257 of 1000.
Processed Record 258 of 1000.
Processed Record 259 of 1000.
Processed Record 260 of 1000.
Wrong Type - Skipped Record 261 of 1000.
Processed Record 262 of 1000.
Wrong Type - Skipped Record 263 of 1000.
Wrong Type - Skipped Record 264 of 1000.
Processed Record 265 of 1000.
Processed Record 266 of 1000.
Wrong Type - Skipped Record 267 of 1000.
Processed Record 268 of 1000.
Processed Record 269 of 1000.
Wrong Type - Skipped Record 270 of 1000.
Wrong Type - Skipped Record 271 of 1000.
Processed Record 272 of 1000.
Processed Record 273 of 1000.
Wrong Type - Skipped Record 274 of 1000.
Processed Record 275 of 1000.
Processed Record 276 of 1000.
Processed Record 277 of 1000.
P

Wrong Type - Skipped Record 492 of 1000.
Processed Record 493 of 1000.
Processed Record 494 of 1000.
Processed Record 495 of 1000.
Wrong Type - Skipped Record 496 of 1000.
Processed Record 497 of 1000.
Processed Record 498 of 1000.
Processed Record 499 of 1000.
Wrong Type - Skipped Record 500 of 1000.
Processed Record 501 of 1000.
Wrong Type - Skipped Record 502 of 1000.
Wrong Type - Skipped Record 503 of 1000.
Processed Record 504 of 1000.
Processed Record 505 of 1000.
Processed Record 506 of 1000.
Processed Record 507 of 1000.
Wrong Type - Skipped Record 508 of 1000.
Wrong Type - Skipped Record 509 of 1000.
Processed Record 510 of 1000.
Wrong Type - Skipped Record 511 of 1000.
Processed Record 512 of 1000.
Wrong Type - Skipped Record 513 of 1000.
Processed Record 514 of 1000.
Processed Record 515 of 1000.
Wrong Type - Skipped Record 516 of 1000.
Processed Record 517 of 1000.
Wrong Type - Skipped Record 518 of 1000.
Wrong Type - Skipped Record 519 of 1000.
Processed Record 520 of 1000

Processed Record 739 of 1000.
Processed Record 740 of 1000.
Processed Record 741 of 1000.
Wrong Type - Skipped Record 742 of 1000.
Processed Record 743 of 1000.
Wrong Type - Skipped Record 744 of 1000.
Processed Record 745 of 1000.
Wrong Type - Skipped Record 746 of 1000.
Processed Record 747 of 1000.
Processed Record 748 of 1000.
Processed Record 749 of 1000.
Wrong Type - Skipped Record 750 of 1000.
Processed Record 751 of 1000.
Wrong Type - Skipped Record 752 of 1000.
Processed Record 753 of 1000.
Processed Record 754 of 1000.
Processed Record 755 of 1000.
Processed Record 756 of 1000.
Processed Record 757 of 1000.
Wrong Type - Skipped Record 758 of 1000.
Wrong Type - Skipped Record 759 of 1000.
Processed Record 760 of 1000.
Processed Record 761 of 1000.
Wrong Type - Skipped Record 762 of 1000.
Processed Record 763 of 1000.
Processed Record 764 of 1000.
Wrong Type - Skipped Record 765 of 1000.
Processed Record 766 of 1000.
Processed Record 767 of 1000.
Processed Record 768 of 1000.
W

Processed Record 983 of 1000.
Processed Record 984 of 1000.
Processed Record 985 of 1000.
Processed Record 986 of 1000.
Wrong Type - Skipped Record 987 of 1000.
Processed Record 988 of 1000.
Processed Record 989 of 1000.
Wrong Type - Skipped Record 990 of 1000.
Processed Record 991 of 1000.
Processed Record 992 of 1000.
Wrong Type - Skipped Record 993 of 1000.
Wrong Type - Skipped Record 994 of 1000.
Processed Record 995 of 1000.
Processed Record 996 of 1000.
Processed Record 997 of 1000.
Processed Record 998 of 1000.
Wrong Type - Skipped Record 999 of 1000.
Processed Record 1000 of 1000.


In [14]:
# Create a dataframe with all addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
#     "type": aType,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
#     "address type": addType
})
address_df

# drop type and address type, or rather don't gather in the first place

,house #,street,zipcode,lat,lon,neighborhood
0,4704,Carter Lane,78744,30.205996,-97.7352885531609,Southeast Austin
1,9302,Creeks Edge Circle,78733,30.295867,-97.8844720587302,NA
2,5350,West US Highway 290,TX 78735,30.2372955,-97.8375895227211,Sunset Oaks
3,1404,Green Pastures Cove,78725,30.222968,-97.6283230819783,Garden Valley Village
4,7308,Carver Avenue,78752,30.3325235,-97.698402548495,St. Johns
5,1203,Cometa Street,78721,30.278904,-97.68943,MLK
6,3955,Westlake Drive,78731,30.3289255,-97.7807829791667,Balcones Park
7,9909,Thaxton Road,78747,30.130941,-97.735565,Thaxton Place
8,1200,Capital of Texas Highway,78746,30.281526,-97.823011,Ledgeway
9,3242,Lockhart Highway,78744,30.20248564,-97.68332272,NA


In [15]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [16]:
# Pull from CSV
address_sample = pd.read_csv(addressList_csv)
address_sample.head()

,house #,street,zipcode,lat,lon,neighborhood
0,4704,Carter Lane,78744,30.205996,-97.735289,Southeast Austin
1,9302,Creeks Edge Circle,78733,30.295867,-97.884472,NaN
2,5350,West US Highway 290,TX 78735,30.237296,-97.837590,Sunset Oaks
3,1404,Green Pastures Cove,78725,30.222968,-97.628323,Garden Valley Village
4,7308,Carver Avenue,78752,30.332523,-97.698403,St. Johns


In [21]:
# Map out the sample with gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(address_sample["lat"], address_sample["lon"], 'red', size=80, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [18]:
# Val create function for read in csv, check for headers?, and append csv


# masterData.csv

In [19]:
address = address_df["house #"][1] + " " + address_df["street"][1] 
zipcode = address_df["zipcode"][1]

print(address)
print(zipcode)

pp = pprint.PrettyPrinter(indent=4)

api = zillow.ValuationApi()
# Insert your Zillow API key here
# zwsid = ""

#This appears to be the ideal search to use with the most flushed out info
deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
pp.pprint(deep_results.get_dict())


# This appears to return the same as .GetZEstimate(),
# but has less accurate "extended_data" than .GetDeepSearchResults
# data = api.GetSearchResults(zwsid, address, zipcode)
# pp.pprint(data.get_dict())


# This appears to return the same as .GetSearchResults()
# detail_data = api.GetZEstimate(zwsid, data.zpid)
# pp.pprint(detail_data.get_dict())

9302 Creeks Edge Circle
78733


NameError: name 'zwsid' is not defined

## Calculate Value per Sqft

In [ ]:
#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]


## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section